In [1]:
# Librerias
import os
import re
import time
import glob
import json
import random
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC

In [2]:
archivo_rucs = "ListaRucs.txt"
archivo_procesados = 'RucsProcesados.txt'

def cargar_rucs(path):
    if os.path.exists(path):
        with open(path, 'r', encoding='utf-8') as f:
            return set(line.strip() for line in f if line.strip())
    return set()

Rucs = cargar_rucs(archivo_rucs)
Rucs = pd.DataFrame(list(Rucs), columns=['Rucs'])
RucProcesados = cargar_rucs(archivo_procesados)

Rucs = Rucs[~Rucs['Rucs'].astype(str).isin(set(RucProcesados))]
Rucs = Rucs[Rucs['Rucs'].astype(str).str.match(r'^\d{11}$')].reset_index(drop=True)

print(f"Faltan procesar {len(Rucs)} Rucs")

Faltan procesar 0 Rucs


In [3]:
rutaOesce = os.path.join(os.getcwd(), 'RegistroOece')
for index, row in Rucs.iterrows():
    ruc = str(row['Rucs'])
    print(f"Obteniendo HTML para el RUC: {ruc}")

    URL = "https://eap.oece.gob.pe/perfilprov-bus/1.0/ficha/" + ruc + "/contrataciones/exportar"

    time.sleep(10)

    driver = webdriver.Chrome()
    driver.get(URL)
    
    time.sleep(5)
    
    html_completo = driver.page_source
    nombre_archivo = f'procesado_{ruc}.html'
    ruta_archivo = os.path.join(rutaOesce, nombre_archivo)

    with open(rutaOesce, 'w', encoding='utf-8') as f:
        f.write(html_completo)

    with open(archivo_procesados, 'a', encoding='utf-8') as f:
        f.write(f"{ruc}\n")

    try:
        driver.quit()
    except Exception as e:
        print(f"Error al cerrar el navegador para el RUC {ruc}: {e}")

In [4]:
rutaRuc = os.path.join(os.getcwd(), 'RegistroRuc')
archivos_actuales = [f for f in os.listdir(rutaRuc)]
archivo_imposibles = 'RucsImposibles.txt'
archivo_rucs = "ListaRucs.txt"

def cargar_rucs(path):
    if os.path.exists(path):
        with open(path, 'r', encoding='utf-8') as f:
            return set(line.strip() for line in f if line.strip())
    return set()

Rucs = cargar_rucs(archivo_rucs)
Rucs = pd.DataFrame(list(Rucs), columns=['Rucs'])

RucImposibles = cargar_rucs(archivo_imposibles)
RucProcesados = [archivo[10:-5] for archivo in archivos_actuales]

Rucs = Rucs[~Rucs['Rucs'].astype(str).isin(set(RucProcesados) | set(RucImposibles))]
Rucs = Rucs[Rucs['Rucs'].astype(str).str.match(r'^\d{11}$')].reset_index(drop=True)

print(f"Faltan procesar {len(Rucs)} Rucs")

Faltan procesar 0 Rucs


In [5]:
Rucs_imposibles = []

for index, row in Rucs.iterrows():
    ruc = str(row['Rucs'])
    print(f"Obteniendo HTML para el RUC: {ruc}")

    time.sleep(10)

    driver = webdriver.Chrome()
    driver.get("https://e-consultaruc.sunat.gob.pe/cl-ti-itmrconsruc/FrameCriterioBusquedaWeb.jsp")
    
    time.sleep(5)
    
    ruc_input = driver.find_element(By.ID, 'txtRuc')
    ruc_input.clear()
    ruc_input.send_keys(ruc)

    time.sleep(5)

    buscar_button = driver.find_element(By.ID, 'btnAceptar')
    buscar_button.click()

    try:
        alert = WebDriverWait(driver, 5).until(EC.alert_is_present())
        alert_text = alert.text
        alert.accept()

        if "ingrese número de RUC válido" in alert_text.lower():
            Rucs_imposibles.append(ruc)
            print(f"RUC inválido: {ruc} (alerta detectada)")
            driver.quit()
            continue 

    except TimeoutException:
        pass

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//h4[contains(text(), 'Fecha de Inscripción')]")))
    time.sleep(2)

    html_completo = driver.page_source
    nombre_archivo = f'procesado_{ruc}.html'
    ruta_archivo = os.path.join(rutaRuc, nombre_archivo)

    with open(ruta_archivo, 'w', encoding='utf-8') as f:
        f.write(html_completo)

    try:
        driver.quit()
    except Exception as e:
        print(f"Error al cerrar el navegador para el RUC {ruc}: {e}")

if Rucs_imposibles:
    with open(archivo_imposibles, 'a', encoding='utf-8') as f:
        for ruc in Rucs_imposibles:
            f.write(f"{ruc}\n")

In [6]:
def extraer_datos(html_completo):
    soup = BeautifulSoup(html_completo, 'html.parser')
    
    data = {}
    
    ruc_info = soup.find('h4', class_='list-group-item-heading').find_next('h4').text.strip()
    data['Número de RUC'] = ruc_info.split(' - ')[0]
    data['Razón Social'] = ruc_info.split(' - ')[1]
    
    data['Tipo Contribuyente'] = soup.find(text="Tipo Contribuyente:").find_next('p').text.strip()
    nombre_comercial = soup.find(text="Nombre Comercial:").find_next('p').text.strip()
    data['Nombre Comercial'] = nombre_comercial if nombre_comercial != '-' else None
    
    data['Fecha de Inscripción'] = soup.find(text="Fecha de Inscripción:").find_next('p').text.strip()
    data['Fecha de Inicio de Actividades'] = soup.find(text="Fecha de Inicio de Actividades:").find_next('p').text.strip()
    
    data['Estado del Contribuyente'] = soup.find(text="Estado del Contribuyente:").find_next('p').text.strip()
    data['Condición del Contribuyente'] = soup.find(text="Condición del Contribuyente:").find_next('p').text.strip()
    
    data['Domicilio Fiscal'] = soup.find(text="Domicilio Fiscal:").find_next('p').text.strip()
    
    data['Sistema Emisión de Comprobante'] = soup.find(text="Sistema Emisión de Comprobante:").find_next('p').text.strip()
    data['Actividad Comercio Exterior'] = soup.find(text="Actividad Comercio Exterior:").find_next('p').text.strip()
    
    data['Sistema Contabilidad'] = soup.find(text="Sistema Contabilidad:").find_next('p').text.strip()
    
    actividad_economica = soup.find(text="Actividad(es) Económica(s):").find_next('table').find_all('td')
    data['Actividad Económica'] = [act.text.strip() for act in actividad_economica]
    
    comprobantes_pago = soup.find(text="Comprobantes de Pago c/aut. de impresión (F. 806 u 816):").find_next('table').find_all('td')
    data['Comprobantes de Pago'] = [comp.text.strip() for comp in comprobantes_pago]
    
    data['Emisor electrónico desde'] = soup.find(text="Emisor electrónico desde:").find_next('p').text.strip()
    
    data['Comprobantes Electrónicos'] = soup.find(text="Comprobantes Electrónicos:").find_next('p').text.strip()
    
    data['Afiliado al PLE desde'] = soup.find(text="Afiliado al PLE desde:").find_next('p').text.strip()
    
    padrones = soup.find(text="Padrones:").find_next('table').find_all('td')
    data['Padrones'] = [padron.text.strip() for padron in padrones]
    
    return data

In [7]:
data_final = []

for archivo in os.listdir(rutaRuc):
    if archivo.endswith('.html') and archivo.startswith('procesado_'):
        ruta_archivo = os.path.join(rutaRuc, archivo)
        with open(ruta_archivo, 'r', encoding='utf-8') as f:
            html_completo = f.read()
        
        try:
            data = extraer_datos(html_completo)
            data_final.append(data)
        except Exception as e:
            print(f"Error procesando {archivo}: {e}")

df_final = pd.DataFrame(data_final)
#df_final.to_parquet('Rucs.parquet', index=False)

C:\Users\eholgado\AppData\Local\Temp\ipykernel_28168\1797155849.py:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  data['Tipo Contribuyente'] = soup.find(text="Tipo Contribuyente:").find_next('p').text.strip()
C:\Users\eholgado\AppData\Local\Temp\ipykernel_28168\1797155849.py:11: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  nombre_comercial = soup.find(text="Nombre Comercial:").find_next('p').text.strip()
C:\Users\eholgado\AppData\Local\Temp\ipykernel_28168\1797155849.py:14: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  data['Fecha de Inscripción'] = soup.find(text="Fecha de Inscripción:").find_next('p').text.strip()
C:\Users\eholgado\AppData\Local\Temp\ipykernel_28168\1797155849.py:15: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  data['Fecha de Inicio de Activi

In [8]:
rutaOesce = os.path.join(os.getcwd(), 'RegistroOece')
os.makedirs(rutaOesce, exist_ok=True)

lista_dfs = []

for archivo in os.listdir(rutaOesce):
    if archivo.endswith('.html'):
        ruta_archivo = os.path.join(rutaOesce, archivo)
        with open(ruta_archivo, 'r', encoding='utf-8') as f:
            soup = BeautifulSoup(f, 'html.parser')
            data_json = json.loads(soup.find('pre').text)
            if 'contratosE01' in data_json:
                df = pd.DataFrame(data_json['contratosE01'])
                ruc = archivo.replace('procesado_', '').replace('.html', '')
                df['RUC Origen'] = ruc
                lista_dfs.append(df)

df_consolidado = pd.concat(lista_dfs, ignore_index=True)
df_consolidado = df_consolidado.drop_duplicates()

In [9]:
df_consolidado['miembros_consorcio'] = df_consolidado['miembros_consorcio'].fillna('')  # reemplaza NaN por ''
mask_vacio = df_consolidado['miembros_consorcio'].str.strip() == ''  # filas con cadena vacía o espacios
df_consolidado.loc[mask_vacio, 'miembros_consorcio'] = df_consolidado.loc[mask_vacio, 'RUC Origen'].astype(str) + "|100"

In [10]:
def obtener_porcentaje(ruc_objetivo, miembros):
    if pd.isna(miembros):
        return None
    pares = miembros.split('||')
    for par in pares:
        partes = par.split('|')
        if len(partes) >= 2 and partes[0] == ruc_objetivo:
            try:
                return float(partes[1])
            except ValueError:
                return None
    return None

df_consolidado['monto_del_contrato_original'] = pd.to_numeric(
    df_consolidado['monto_del_contrato_original'], errors='coerce')

df_consolidado['participacion_ruc_origen'] = df_consolidado.apply(
    lambda row: obtener_porcentaje(row['RUC Origen'], row.get('miembros_consorcio')),
    axis=1)

df_consolidado['Valor Proporcional GE'] = df_consolidado['participacion_ruc_origen'] * df_consolidado['monto_del_contrato_original'] /100

df_consolidado['fecha_de_firma_de_contrato'] = pd.to_datetime(df_consolidado['fecha_de_firma_de_contrato'], dayfirst=True, errors='coerce')
df_consolidado['fecha_prevista_de_fin_de_contrato'] = pd.to_datetime(df_consolidado['fecha_prevista_de_fin_de_contrato'], dayfirst=True, errors='coerce')

df_consolidado['Nro de dias'] = (
    df_consolidado['fecha_prevista_de_fin_de_contrato'] - df_consolidado['fecha_de_firma_de_contrato']).dt.days

df_consolidado['Valor Mensual proporcional'] = (df_consolidado['Valor Proporcional GE']/df_consolidado['Nro de dias'])*30

df_consolidado['plazo en Meses'] = round(df_consolidado['Nro de dias'] /30,1)

df_consolidado['Valor Proporcional GE'] = df_consolidado['Valor Proporcional GE'].apply(
    lambda x: f"{x:,.2f}" if pd.notna(x) else "")

df_consolidado['Valor Mensual proporcional'] = df_consolidado['Valor Mensual proporcional'].apply(
    lambda x: f"{x:,.2f}" if pd.notna(x) else "")

In [11]:
df_consolidado.rename(columns={
    'objeto': 'Objeto',
    'descripcion': 'Descripción',
    'entidad': 'Entidad',
    'moneda_del_monto_del_contrato_original': 'Moneda del Contrato Original',
    'monto_del_contrato_original': 'Monto del Contrato Original',
    'fecha_de_firma_de_contrato': 'Fecha de Firma de Contrato',
    'fecha_prevista_de_fin_de_contrato': 'Fecha Prevista de FIn de Contrato',
    'miembros_consorcio': 'Miembros Consorcio',
    'estado': 'Estado',
    'RUC Origen': 'RUC',
    'participacion_ruc_origen': '% Participación RUC',
    'Valor Proporcional GE': 'Valor Proporcional GE',
    'Nro de dias': 'N° Días',
    'Valor Mensual proporcional': 'Valor Mensual proporcional',
    'plazo en Meses': 'Plazo en Meses'
}, inplace=True)


In [12]:
df_final.rename(columns={'Número de RUC': 'RUC'}, inplace=True)

df_consolidado['RUC'] = df_consolidado['RUC'].astype(str).str.strip()
df_final['RUC'] = df_final['RUC'].astype(str).str.strip()

df_consolidado = pd.merge(
    df_consolidado,
    df_final[['RUC', 'Razón Social']],
    on='RUC',
    how='left')

In [13]:
df_consolidado['RUC'] = df_consolidado['RUC'].astype(str).str.strip()

df_consolidado['Fecha de Firma de Contrato'] = pd.to_datetime(df_consolidado['Fecha de Firma de Contrato'], errors='coerce')
df_consolidado['Fecha Prevista de FIn de Contrato'] = pd.to_datetime(df_consolidado['Fecha Prevista de FIn de Contrato'], errors='coerce')

df_consolidado['% Participación RUC'] = pd.to_numeric(df_consolidado['% Participación RUC'], errors='coerce')

df_consolidado['Valor Proporcional GE'] = df_consolidado['Valor Proporcional GE'].astype(str).str.replace(',', '')
df_consolidado['Valor Proporcional GE'] = pd.to_numeric(df_consolidado['Valor Proporcional GE'], errors='coerce')

df_consolidado['Monto del Contrato Original'] = df_consolidado['Monto del Contrato Original'].astype(str).str.replace(',', '')
df_consolidado['Monto del Contrato Original'] = pd.to_numeric(df_consolidado['Monto del Contrato Original'], errors='coerce')

df_consolidado['N° Días'] = round(df_consolidado['N° Días'],2)

In [14]:
cols = ['RUC', 'Razón Social'] + [col for col in df_consolidado.columns if col not in ['RUC', 'Razón Social']]
df_consolidado = df_consolidado[cols]

In [15]:
df_consolidado.to_parquet("Consolidado.parquet", index=False)

In [16]:
df_consolidado = pd.read_parquet("Consolidado.parquet")

df_consolidado['Valor Mensual proporcional'] = pd.to_numeric(
    df_consolidado['Valor Mensual proporcional'].astype(str).str.replace(',', ''),
    errors='coerce'
).fillna(0)

fecha_inicio = df_consolidado['Fecha de Firma de Contrato'].min()
fecha_fin = df_consolidado['Fecha Prevista de FIn de Contrato'].max()
fechas_periodos = pd.date_range(start=fecha_inicio, end=fecha_fin, freq='M')

registro = []

for periodo in fechas_periodos:
    contratos_activos = df_consolidado[
        (df_consolidado['Fecha de Firma de Contrato'] <= periodo) &
        (df_consolidado['Fecha Prevista de FIn de Contrato'] >= periodo)
    ].copy()
    
    contratos_activos['Fecha Periodo'] = periodo
    contratos_activos['Periodo'] = periodo.strftime('%B %Y')
    contratos_activos = contratos_activos[['RUC', 'Razón Social', 'Periodo', 'Fecha Periodo', 'Valor Mensual proporcional']]
    
    registro.append(contratos_activos)

df_mensual = pd.concat(registro, ignore_index=True)

df_resumen = df_mensual.groupby(
    ['RUC', 'Razón Social', 'Periodo', 'Fecha Periodo'],
    as_index=False
)['Valor Mensual proporcional'].sum()

df_resumen['Valor Mensual proporcional'] = df_resumen['Valor Mensual proporcional'].fillna(0)
df_resumen.to_parquet("DataMensualContrato.parquet", index=False)

C:\Users\eholgado\AppData\Local\Temp\ipykernel_28168\157627733.py:10: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  fechas_periodos = pd.date_range(start=fecha_inicio, end=fecha_fin, freq='M')
